<a href="https://colab.research.google.com/github/turtlenoise/simple_sentiment_analysis/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import thinc.extra.datasets

imdb_data = thinc.extra.datasets.imdb()

sentences, labels = zip(*imdb_data[1])

In [3]:
print(str(labels[0]) + " " + sentences[0])

0 this movie is so bad and Hellraiser part 1 to 3 are so great. Nothing is good Bloodline... a lot of gore but without meaning and the majority of time without originality. the movie is about the past, the present (1996) and the future of the puzzle box. The creator of the box and his offspring fight against some demon but nothing link up seriously the 3 stories. Pinhead is in the movie...but it isn't Pinhead, he is just a boring bad guy who kidnap kids and kill bird, and the worse...he talk too much. A thing that I really didn't understand is why the dog from hell????? He can't be a cenobites because he had nothing human. Why the dog can be killed by the pressure if the other creature from hell can receive bullets without problems????? Bloodline is incoherent with the movie series in many ways. For example Pinhead and Angelique are old friend in this movie but it's impossible that Pinhead met Angelique because Pinhead is just a soldier from the First World War and Angelique is an old 

In [19]:
negative_text = ""
for i in range(1,1500):
  if (labels[i] == 0):
    negative_text = negative_text + sentences[i]

In [20]:
import spacy
nlp = spacy.load('en_core_web_sm')
corpus = nlp(negative_text)

In [41]:
def has_negative_children(token):
  for child in token.children:
    if (child.dep_ == 'neg'):
      return True
  return False 

negative_adverb_list = []
negative_adjective_list = []
negative_word_frequency = dict()
for sent in corpus.sents:
    for token in sent:
        if (token.lemma_ in negative_word_frequency.keys()) and (has_negative_children(token) == False):
            negative_word_frequency[token.lemma_] += 1
        else:
            negative_word_frequency[token.lemma_] = 1
        if (token.pos_ == 'ADJ') and (has_negative_children(token) == False):
          negative_adjective_list.append(token.lemma_)
        if (token.pos_ == "ADV") and (has_negative_children(token) == False):
          negative_adverb_list.append(token.lemma_)

In [44]:
negative_word_frequency['bad']

140

In [52]:
negative_word_frequency['good']

8

In [45]:
positive_text = ""
for i in range(1,1500):
  if (labels[i] == 1):
    positive_text = positive_text + sentences[i]

In [46]:
corpus = nlp(positive_text)

In [61]:
positive_adverb_list = []
positive_adjective_list = []
positive_word_frequency = dict()
for sent in corpus.sents:
    for token in sent:
        if (token.lemma_ in positive_word_frequency.keys()) and (has_negative_children(token) == False):
            positive_word_frequency[token.lemma_] += 1
        else:
            positive_word_frequency[token.lemma_] = 1
        if (token.pos_ == 'ADJ') and (has_negative_children(token) == False):
          positive_adjective_list.append(token.lemma_)
        if (token.pos_ == 'ADV') and (has_negative_children(token) == False):
          positive_adverb_list.append(token.lemma_)

In [51]:
positive_word_frequency['good']

261

In [53]:
positive_word_frequency['bad']

44

In [56]:
word_values = dict()
for word in negative_word_frequency:
  if (word not in word_values):
    word_values[word] = negative_word_frequency[word]

for word in positive_word_frequency:
  if (word in word_values):
    word_values[word] = positive_word_frequency[word] / (positive_word_frequency[word] + negative_word_frequency[word])
  else: 
    word_values[word] = 1

In [57]:
word_values['good']

0.9702602230483272

In [58]:
word_values['lousy']

0.16666666666666666

In [59]:
word_values['blue']

0.6086956521739131

In [98]:
def get_score(example_sentence):
  corpus = nlp(example_sentence)
  sum = 0
  count = 0
  for sent in corpus.sents:
      for token in sent:
        if (token.lemma_ in word_values) and (token.pos_ == 'ADJ' or token.pos_ == 'ADV'):
          sum = sum + 2*word_values[token.lemma_]
          count = count + 2
        if (token.lemma_ in word_values):
          sum = sum + word_values[token.lemma_]
          count = count + 1
  average_sentiment = sum/count
  return average_sentiment


In [102]:
example_sentence = "Best movie I have ever seen, amazing job, the director is a genius."
get_score(example_sentence)

0.6621121985153715

In [100]:
example_sentence = "Scary, boring, totally disgusting, yucks."
get_score(example_sentence)

0.4402968060943681

In [101]:
example_sentence = "Best movie I have ever seen although it had some scary, boring and disgusting scenes."
get_score(example_sentence)

0.5449916275254358